In [1]:
%load_ext jupyter_black
%reload_ext jupyter_black

# Aggiornamento di un dataset con clear ml
Supponiamo che vogliamo fare un aggiornamento di un dataset con clear ml. In particolare, supponiamo di voler aggiornare un dataset con delle immagini ridimensionate perchè il dataset presenta immagini di dimensioni diverse e anche molto grandi per un modello di  Deep Learning.  In questo potremo usare la libreria clearml per fare l'aggiornamento del dataset per noi e per il resto del team, in modo che tutti possano avere accesso al dataset aggiornato e non devono stare a fare un preprocessing delle immagini già fatte.

In [2]:
# %env CLEARML_WEB_HOST=https://app.clear.ml
# %env CLEARML_API_HOST=https://api.clear.ml
# %env CLEARML_FILES_HOST=https://files.clear.ml
# %env CLEARML_API_ACCESS_KEY=C30F7MH4O8PLDDUCBVOD
# %env CLEARML_API_SECRET_KEY=5J64DUYTvA6aiWfVxAU01TUiCBJBqfepaGmxXCcw9TTyJBhM3Y
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=ZQ2HO9ZWN2AD2VF1ZAJI
%env CLEARML_API_SECRET_KEY=eUpHz1AWvp5vsqLOYWVMwt13bnJg0TFEbfyrYzUr1XqNFemTfI

env: CLEARML_WEB_HOST=https://app.clear.ml
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=ZQ2HO9ZWN2AD2VF1ZAJI
env: CLEARML_API_SECRET_KEY=eUpHz1AWvp5vsqLOYWVMwt13bnJg0TFEbfyrYzUr1XqNFemTfI


In [3]:
from pathlib import Path
import cv2
from clearml import Task, Dataset, TaskTypes
import matplotlib.pyplot as plt

# Creazione di un task
Anche qui creiamo un task per poter la creazione di questo preprocessing delle immagini.

In [4]:
task = Task.init(project_name="fruit_classification_with_vit", task_name="pre_process_data", task_type= TaskTypes.data_processing)

ClearML Task: created new task id=c8c2316797884cb1aa6910013b32c878
2024-02-09 00:04:19,327 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/c2985cd7c0bf4c179801d7f86acd0ab6/experiments/c8c2316797884cb1aa6910013b32c878/output/log


# Creazione di una funzione per il preprocessing delle immagini
Creiamo una funzione che ci permette di fare il preprocessing delle immagini. In particolare, questa funzione ci permette di fare il ridimensionamento delle immagini.

In [5]:
def load_image(image_path:str):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

In [6]:
def show_image(image):
    plt.imshow(image)
    plt.axis('off')
    plt.show()

In [7]:
def save_image(image, image_path:str):
    cv2.imwrite(image_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

In [8]:
def resize_image(image, size:int):
    image_smooth = cv2.GaussianBlur(image, (7, 7), 0)
    return cv2.resize(image_smooth, (size, size))

In [9]:
path_to_load_data = Path("..", "data", "fruit_dataset")
path_to_save_data = Path("..", "data", "fruit_dataset_resized")

In [10]:
folders = []
counts = []
for class_folder in path_to_load_data.iterdir():
    if class_folder.is_dir():
        name_folder = class_folder.name
        folders.append(name_folder)
        path_to_save_class = path_to_save_data / name_folder
        path_to_save_class.mkdir(exist_ok=True, parents=True)
        count = 0
        for image_path in class_folder.iterdir():
            if image_path.suffix != '.gif':
                #print(image_path.as_posix())
                count += 1
                image = load_image(image_path.as_posix())
                image_resized = resize_image(image, 224)
                save_image(image_resized, (path_to_save_data / class_folder.name / image_path.name).as_posix())
        counts.append([count])

libpng warning: iCCP: known incorrect sRGB profile


In [11]:
dataset = Dataset.create(dataset_name="fruit_dataset_resized", dataset_project="fruit_classification_with_vit", parent_datasets=['e20bc7d4e64343d092a16d056866cffd'])


2024-02-09 00:04:39,360 - clearml - INFO - Dataset.get() did not specify alias. Dataset information will not be automatically logged in ClearML Server.
2024-02-09 00:04:39,919 - clearml.Task - ERROR - Action failed <400/101: tasks.get_by_id/v1.0 (Invalid task id: ids=(e20bc7d4e64343d092a16d056866cffd))> (task=e20bc7d4e64343d092a16d056866cffd)
2024-02-09 00:04:39,921 - clearml.Task - ERROR - Failed reloading task e20bc7d4e64343d092a16d056866cffd
2024-02-09 00:04:40,470 - clearml.Task - ERROR - Action failed <400/101: tasks.get_by_id/v1.0 (Invalid task id: ids=(e20bc7d4e64343d092a16d056866cffd))> (task=e20bc7d4e64343d092a16d056866cffd)
2024-02-09 00:04:40,471 - clearml.Task - ERROR - Failed reloading task e20bc7d4e64343d092a16d056866cffd


ValueError: Task ID "e20bc7d4e64343d092a16d056866cffd" could not be found

In [ ]:
dataset.sync_folder(local_path=path_to_save_data.as_posix(), verbose=True)

In [ ]:
dataset.get_logger().report_histogram(
    title="Number of images per class",
    series="Number of images",
    labels=folders,
    values=counts,
    xaxis="Class",
    yaxis="Count"
)

In [ ]:
dataset.finalize(verbose=True, auto_upload=True)
task.close()